# Exact solution used in MES runs


We would like to MES the operation

\begin{align*}
    \iint \mathbf{v} \cdot \text{d}\mathbf{S}
\end{align*}

Where $\text{d}\mathbf{S}$ is the edge of the domain. That is

\begin{align*}
    -
    \int_0^{Lx} \int_0^{2\pi} \mathbf{v} \cdot \mathbf{e}^y J\text{d}\theta\text{d}\rho
    =
    -
    \int_0^{Lx} \int_0^{2\pi} v^y J\text{d}\theta\text{d}\rho
    \qquad &\text{Lower circular disk}
    \\
    \int_0^{Lx} \int_0^{2\pi} \mathbf{v} \cdot \mathbf{e}^y J\text{d}\theta\text{d}\rho
    =
    \int_0^{Lx} \int_0^{2\pi} v^y J\text{d}\theta\text{d}\rho
    \qquad &\text{Upper circular disk}
    \\
    \int_0^{Ly} \int_0^{2\pi} \mathbf{v} \cdot \mathbf{e}^{\rho} J \text{d}\theta\text{d}y
    =
    \int_0^{Ly} \int_0^{2\pi} v^{\rho} J \text{d}\theta\text{d}y
    \qquad &\text{Outer x surface}
    \\
\end{align*}

Using cylindrical geometry.

We will here specify the vector using contravariant components.

In [1]:
%matplotlib notebook
import numpy as np

from sympy import init_printing
from sympy import S
from sympy import sin, cos, tanh, exp, pi, sqrt
from sympy import integrate

from boutdata.mms import x, y, z, t

import os, sys
# If we add to sys.path, then it must be an absolute path
common_dir = os.path.abspath('./../../../')
# Sys path is a list of system paths
sys.path.append(common_dir)
from common.python.mesGenerator import get_metric, BOUT_print

init_printing()

## Initialize

In [2]:
folder = '../sumSines/'
metric = get_metric()

## Define the variables

In [3]:
# Initialization
the_vars = {}

### Define the function to volume integrate

**NOTE**:

1. z must be periodic
2. The field $f(\rho, \theta)$ must be of class infinity in $z=0$ and $z=2\pi$
3. The field $f(\rho, \theta)$ must be single valued when $\rho\to0$
4. The field $f(\rho, \theta)$ must be continuous in the $\rho$ direction with $f(\rho, \theta + \pi)$
5. Eventual BC in $\rho$ must be satisfied

In [4]:
# We need Lx and Ly
from boututils.options import BOUTOptions
myOpts = BOUTOptions(folder)
Lx = eval(myOpts.geom['Lx'])
Ly = eval(myOpts.geom['Ly'])

In [5]:
# Normalized sum of sines
the_vars['v^x'] = (sin(y/Ly + 0.1) + sin(2.0*y/Ly + 1)   + sin(2.5*y/Ly + 0.3) +\
                   sin(z + 2.0)    + sin(2.0*z + 2.1)    + sin(3.0*z))/\
                 12978.4154791334
    
the_vars['v^y'] = (sin(x/Lx + 2.0) + sin(2.0*x/Lx + 2.1) + sin(3.0*x/Lx + 0.5) +\
                   sin(z + 2.0)    + sin(2.0*z + 2.1)    + sin(3.0*z))/\
                 48.5002596046463
    
the_vars['v^z'] = 0.0

Calculating the solution

In [6]:
the_vars['S_yup'] = (
                 integrate(
                    integrate(x*the_vars['v^y'], (x, 0, Lx)), (z, 0, 2*np.pi)
                          )
                    ).evalf()

# Surface pointing in the negative y-direction
the_vars['S_ydown'] = - the_vars['S_yup']

# Will be evaluated at the x-edge
the_vars['S_xout'] = (
                 integrate(
                    integrate(x*the_vars['v^x'], (y, 0, Ly)), (z, 0, 2*np.pi)
                          )
                    ).subs(x, Lx).evalf()

## Print the variables in BOUT++ format

In [7]:
BOUT_print(the_vars, rational=False)